#Tool-calling Agent

This is an auto-generated notebook created by an AI Playground export.

This notebook uses [Mosaic AI Agent Framework](https://docs.databricks.com/generative-ai/agent-framework/build-genai-apps.html) to recreate your agent from the AI Playground. It  demonstrates how to develop, manually test, evaluate, log, and deploy a tool-calling agent in LangGraph.

The agent code implements [MLflow's ChatAgent](https://mlflow.org/docs/latest/python_api/mlflow.pyfunc.html#mlflow.pyfunc.ChatAgent) interface, a Databricks-recommended open-source standard that simplifies authoring multi-turn conversational agents, and is fully compatible with Mosaic AI agent framework functionality.

 **_NOTE:_**  This notebook uses LangChain, but AI Agent Framework is compatible with any agent authoring framework, including LlamaIndex or pure Python agents written with the OpenAI SDK.

## Prerequisites

- Address all `TODO`s in this notebook.

In [0]:
%pip install -U -qqqq mlflow langchain langgraph==0.3.4 databricks-langchain pydantic databricks-agents unitycatalog-langchain[databricks] uv
%pip install -U langchain-nimble langchain-openai
%pip install -U -qqqq mlflow backoff databricks-openai databricks-agents uv
%pip install -U -qqqq mlflow databricks-langchain databricks-agents uv langgraph==0.3.4
%pip install -U -qqqq langchain_mcp_adapters
%pip install -U -qqqq mlflow langchain langgraph==0.3.4 pydantic unitycatalog-langchain[databricks] uv
%pip install -U -q openai mlflow databricks-agents
%restart_python
dbutils.library.restartPython()

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-server 1.23.4 requires anyio<4,>=3.1.0, but you have anyio 4.9.0 which is incompatible.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.2/65.2 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 730.2/730.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 10.8 MB/s eta 0:00:00
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packa

## Define the agent in code
Below we define our agent code in a single cell, enabling us to easily write it to a local Python file for subsequent logging and deployment using the `%%writefile` magic command.

For more examples of tools to add to your agent, see [docs](https://docs.databricks.com/generative-ai/agent-framework/agent-tool.html).

## Test the agent

Interact with the agent to test its output. Since this notebook called `mlflow.langchain.autolog()` you can view the trace for each step the agent takes.

Replace this placeholder input with an appropriate domain-specific example for your agent.

In [0]:
dbutils.library.restartPython()

In [0]:
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from langchain_core.runnables import RunnableLambda

# === STEP 1: Load your agent1 (tool-calling LangGraphChatAgent) ===
from agent1 import agent as agent1_graph  # already compiled in agent1.py

# === STEP 2: Load your agent2 (ReAct agent created using create_react_agent) ===
from agent2 import load_agent
agent2_react = await load_agent()
# from agent2 import agent as agent2_react  # agent2.py exports a create_react_agent

# === STEP 3: Define shared state ===
class AgentState(TypedDict):
    user_input: str
    prompt_query: Optional[str]
    final_output: Optional[str]

# === STEP 4: Wrap agent1 execution as a node ===
def run_agent1(state: AgentState) -> AgentState:
    print("[Agent 1] Running tool-calling agent to generate structured query...")
    response = agent1_graph.invoke({"messages": [{"role": "user", "content": state["user_input"]}]})
    extracted_message = response["messages"][-1]["content"]
    return {"user_input": state["user_input"], "prompt_query": extracted_message}

# === STEP 5: Wrap agent2 execution as a node ===
async def run_agent2(state: AgentState) -> AgentState:
    print(f"[Agent 2] Running ReAct agent with prompt: {state['prompt_query']}")
    response = await agent2_react.ainvoke({
        "messages": f"Find information about: {state['prompt_query']}. Provide a concise summary."
    })
    # await agent2_react.ainvoke({
    #     "messages": f"Find information about {query} and provide a concise summary."
    # })
    return {
        "user_input": state["user_input"],
        "prompt_query": state["prompt_query"],
        "final_output": response.content
    }

# === STEP 6: Build LangGraph chain ===
builder = StateGraph(AgentState)
builder.add_node("ParseQueryWithAgent1", RunnableLambda(run_agent1))
builder.add_node("ExecuteSearchWithAgent2", RunnableLambda(run_agent2))

builder.set_entry_point("ParseQueryWithAgent1")
builder.add_edge("ParseQueryWithAgent1", "ExecuteSearchWithAgent2")
builder.add_edge("ExecuteSearchWithAgent2", END)

graph = builder.compile()


In [0]:
user_input = """
I'm 33 weeks pregnant, staying in downtown LA. I need to visit a clinic at 10am, and afterward, I’d love to have brunch nearby — maybe pregnancy-safe or Mexican food.
"""

result = await graph.ainvoke({"user_input": user_input})
print(result["final_output"])


[Agent 1] Running tool-calling agent to generate structured query...
[Agent 2] Running ReAct agent with prompt: Based on your request, here's the outline for your needs:

**Recommendation Request:**
"Recommend me top 3 brunch restaurants within 3 miles of downtown LA, available after 11:00 AM, specializing in pregnancy-safe Mexican cuisine or with the highest pregnancy food safety ratings."

**Detailed Breakdown:**

1. **Time Requirement:** After 11:00 AM (allowing time for your 10 AM clinic appointment)

2. **Location Requirement:** Within 3 miles of downtown Los Angeles (easily accessible after your clinic visit)

3. **Place Type:** Restaurant/brunch establishment

4. **Other Requirements:** 
   - Pregnancy-safe food options (avoiding raw/undercooked items, unpasteurized products, high-mercury fish)
   - Mexican cuisine preference OR establishments with excellent pregnancy food safety protocols
   - Suitable for 33-week pregnant woman (comfortable seating, clean facilities, accommoda

ERROR:mcp.client.sse:Error in post_writer: 


[Trace(request_id=tr-c8a5d95012b64da3be2109b60674d8d9), Trace(request_id=tr-ccdcd52feebf49baa561c433ede7637e), Trace(request_id=tr-a126a2f8e37842238504a78e88dc65e3)]

---------------------------------------------------------------------------
AttributeError                            Traceback (most recent call last)
File <command-5922834898495214>, line 5
      1 user_input = """
      2 I'm 33 weeks pregnant, staying in downtown LA. I need to visit a clinic at 10am, and afterward, I’d love to have brunch nearby — maybe pregnancy-safe or Mexican food.
      3 """
----> 5 result = await graph.ainvoke({"user_input": user_input})
      6 print(result["final_output"])

File /local_disk0/.ephemeral_nfs/envs/pythonEnv-2df8a1ff-30ef-4a77-8d72-b9b8d8818ed8/lib/python3.10/site-packages/langgraph/pregel/__init__.py:2420, in Pregel.ainvoke(self, input, config, stream_mode, output_keys, interrupt_before, interrupt_after, debug, **kwargs)
   2418 else:
   2419     chunks = []
-> 2420 async for chunk in self.astream(
   2421     input,
   2422     config,
   2423     stream_mode=stream_mode,
   2424     output_keys=output_keys,
   2425     interrupt_before=inter

In [0]:
for event in AGENT.predict_stream(
    {"messages": [{"role": "user", "content": "What is 5+5 in python"}]}
):
    print(event, "-----------\n")

delta=ChatAgentMessage(role='assistant', content="I notice you're asking about a Python calculation, but I'm specifically designed to help recommend places for pregnant women. \n\nIf you're looking for recommendations for shops, restaurants, or other places that cater to pregnant women's needs, I'd be happy to help. I can suggest locations based on time requirements, location preferences, type of establishment, and specific pregnancy-related needs.\n\nWould you like me to recommend some pregnancy-friendly places in a particular area instead?", name=None, id='run--210dc809-fdba-4883-9862-575d01a7259a-0', tool_calls=None, tool_call_id=None, attachments=None) finish_reason=None custom_outputs=None usage=None -----------



Trace(request_id=tr-11bacc2751314275a10aef27d257e55a)

### Log the `agent` as an MLflow model
Determine Databricks resources to specify for automatic auth passthrough at deployment time
- **TODO**: If your Unity Catalog Function queries a [vector search index](https://docs.databricks.com/generative-ai/agent-framework/unstructured-retrieval-tools.html) or leverages [external functions](https://docs.databricks.com/generative-ai/agent-framework/external-connection-tools.html), you need to include the dependent vector search index and UC connection objects, respectively, as resources. See [docs](https://docs.databricks.com/generative-ai/agent-framework/log-agent.html#specify-resources-for-automatic-authentication-passthrough) for more details.

Log the agent as code from the `agent.py` file. See [MLflow - Models from Code](https://mlflow.org/docs/latest/models.html#models-from-code).

In [0]:
# Determine Databricks resources to specify for automatic auth passthrough at deployment time
import mlflow
from agent import tools, LLM_ENDPOINT_NAME
from databricks_langchain import VectorSearchRetrieverTool
from mlflow.models.resources import DatabricksFunction, DatabricksServingEndpoint
from unitycatalog.ai.langchain.toolkit import UnityCatalogTool

resources = [DatabricksServingEndpoint(endpoint_name=LLM_ENDPOINT_NAME)]
for tool in tools:
    if isinstance(tool, VectorSearchRetrieverTool):
        resources.extend(tool.resources)
    elif isinstance(tool, UnityCatalogTool):
        # TODO: If the UC function includes dependencies like external connection or vector search, please include them manually.
        # See the TODO in the markdown above for more information.
        resources.append(DatabricksFunction(function_name=tool.uc_function_name))

input_example = {
    "messages": [
        {
            "role": "user",
            "content": "I am a 30 year pregnant women. I want to have brunch after my 10am appointment. I will be at 28226 after the appointment. Recommend me safe food brunch choices"
        }
    ]
}

with mlflow.start_run():
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        input_example=input_example,
        resources=resources,
        extra_pip_requirements=[
            "databricks-connect"
        ]
    )

/local_disk0/.ephemeral_nfs/envs/pythonEnv-23b31163-10fe-4763-baaa-9d9b82cbb305/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
2025/06/09 18:14:31 INFO mlflow.pyfunc: Predicting on input example to validate output
/local_disk0/.ephemeral_nfs/envs/pythonEnv-23b31163-10fe-4763-baaa-9d9b82cbb305/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)


Uploading artifacts:   0%|          | 0/12 [00:00<?, ?it/s]

## Evaluate the agent with [Agent Evaluation](https://docs.databricks.com/generative-ai/agent-evaluation/index.html)

You can edit the requests or expected responses in your evaluation dataset and run evaluation as you iterate your agent, leveraging mlflow to track the computed quality metrics.

To evaluate your tool calls, try adding [custom metrics](https://docs.databricks.com/generative-ai/agent-evaluation/custom-metrics.html#evaluating-tool-calls).

In [0]:
import pandas as pd

eval_examples = [
    {
        "request": {
            "messages": [
                {
                    "role": "system",
                    "content": "You are an intelligent AI assistant to recommend best shop, restaurant, and other places for pregnant women to take care about their needs, health and mood. You will take the input from the user and generate an outline with\n1. time requirement (e.g. after 13:00)\n2. location requirement (e.g. within zipcode 28226, 5 miles within a certain address)\n3. place type (e.g. restaurant, shop, medical center)\n4. Other requirement (e.g. safe food choices for pregnant women; medical services specialized for pregnancy)\nBased on the user input, I want the output to be a clear description such as \"Recommend me top 3 brunch within 10 miles of 28226, filter for pregnancy safe or with safest food score.\"\n"
                },
                {
                    "role": "user",
                    "content": "I am a 30 year pregnant women. I want to have brunch after my 10am appointment. I will be at 28226 after the appointment. Recommend me safe food brunch choices"
                }
            ]
        },
        "expected_response": None
    }
]

eval_dataset = pd.DataFrame(eval_examples)
display(eval_dataset)


request,expected_response
"List(List(List(You are an intelligent AI assistant to recommend best shop, restaurant, and other places for pregnant women to take care about their needs, health and mood. You will take the input from the user and generate an outline with 1. time requirement (e.g. after 13:00) 2. location requirement (e.g. within zipcode 28226, 5 miles within a certain address) 3. place type (e.g. restaurant, shop, medical center) 4. Other requirement (e.g. safe food choices for pregnant women; medical services specialized for pregnancy) Based on the user input, I want the output to be a clear description such as ""Recommend me top 3 brunch within 10 miles of 28226, filter for pregnancy safe or with safest food score."" , system), List(I am a 30 year pregnant women. I want to have brunch after my 10am appointment. I will be at 28226 after the appointment. Recommend me safe food brunch choices, user)))",null


In [0]:
import mlflow

with mlflow.start_run(run_id=logged_agent_info.run_id):
    eval_results = mlflow.evaluate(
        f"runs:/{logged_agent_info.run_id}/agent",
        data=eval_dataset,  # Your evaluation dataset
        model_type="databricks-agent",  # Enable Mosaic AI Agent Evaluation
    )

# Review the evaluation results in the MLFLow UI (see console output), or access them in place:
display(eval_results.tables['eval_results'])

/local_disk0/.ephemeral_nfs/envs/pythonEnv-23b31163-10fe-4763-baaa-9d9b82cbb305/lib/python3.10/site-packages/databricks/connect/session.py:454: UserWarning: Ignoring the default notebook Spark session and creating a new Spark Connect session. To use the default notebook Spark session, use DatabricksSession.builder.getOrCreate() with no additional parameters.
  warnings.warn(new_notebook_session_msg)
2025/06/09 18:15:37 INFO mlflow.models.evaluation.utils.trace: Auto tracing is temporarily enabled during the model evaluation for computing some metrics and debugging. To disable tracing, call `mlflow.autolog(disable=True)`.


Evaluating:   0%|          | 0/1 [Elapsed: 00:00, Remaining: ?]

<!DOCTYPE html>
 
 
 Evaluation output 
 
 
 
 
 
 
 
 
 View evaluation results: 1/1 pass

request_id request response trace tool_calls response/overall_assessment/rating response/llm_judged/safety/rating response/llm_judged/safety/rationale response/llm_judged/relevance_to_query/rating response/llm_judged/relevance_to_query/rationale agent/latency_seconds 8a08233be48394cb0275b96edb10d7548d19fcb20630fe5c27d157c5bafa8cea List(List(List(You are an intelligent AI assistant to recommend best shop, restaurant, and other places for pregnant women to take care about their needs, health and mood. You will take the input from the user and generate an outline with
1. time requirement (e.g. after 13:00)
2. location requirement (e.g. within zipcode 28226, 5 miles within a certain address)
3. place type (e.g. restaurant, shop, medical center)
4. Other requirement (e.g. safe food choices for pregnant women; medical services specialized for pregnancy)
Based on the user input, I want the output to be a clear description such as "Recommend me top 3 brunch within 10 miles of 28226, filter for pregnancy safe or with safest food score."
, system), List(I am a 30 year pregnant women. I want to have brunch after my 10am appointment. I will be at 28226 after the appointment. Recommend me safe food brunch choices, user))) List(List(List(Based on your request, here's a clear recommendation outline:

"Recommend me top brunch restaurants within 5 miles of 28226 available after 11:00 AM, prioritizing pregnancy-safe food options with minimal raw ingredients, pasteurized dairy products, and well-cooked proteins."

This search will help you find brunch spots that:
- Are conveniently located near your 28226 appointment location
- Have availability after your 10 AM appointment (allowing travel time)
- Offer menu items that are generally considered safe during pregnancy (fully cooked foods, pasteurized products)
- Provide nutritious options to support your pregnancy health needs, run--e22ecea9-2f95-42a3-b603-15c99f66effd-0, assistant))) {"info": {"request_id": "tr-c415f30a45b748e6b729908a00e495c4", "experiment_id": "3530709779271609", "timestamp_ms": 1749492937484, "execution_time_ms": 6018, "status": "OK", "request_metadata": {"mlflow.sourceRun": "753ab7a05bc945239b0efae3ee16ef3a", "mlflow.traceInputs": "{\"messages\": [{\"role\": \"system\", \"content\": \"You are an intelligent AI assistant to recommend best shop, restaurant, and other places for pregnant women to take care about their needs, health and mood. You will take the input from the user and gen...", "mlflow.traceOutputs": "{\"messages\": [{\"role\": \"system\", \"content\": \"You are an intelligent AI assistant to recommend best shop, restaurant, and other places for pregnant women to take care about their needs, health and mood. You will take the input from the user and gen...", "mlflow.trace_schema.version": "2"}, "tags": {"eval.requestId": "7c604528-49af-4b89-a1da-98cdb4055d5d", "mlflow.databricks.notebook.commandID": "1749492613554_4783097348117565698_17043595fd8842f19f16655f1ebe4df8", "mlflow.databricks.notebookID": "3530709779271609", "mlflow.databricks.notebookPath": "/Users/yuzhang@niagarawater.com/Agent databricks-claude-3-7-sonnet 2025-06-09 11:08:31/driver", "mlflow.databricks.webappURL": "https://dbc-d2dc33b8-a5c3.cloud.databricks.com", "mlflow.databricks.workspaceID": "3750804326266980", "mlflow.databricks.workspaceURL": "https://dbc-d2dc33b8-a5c3.cloud.databricks.com", "mlflow.source.name": "/Users/yuzhang@niagarawater.com/Agent databricks-claude-3-7-sonnet 2025-06-09 11:08:31/driver", "mlflow.source.type": "NOTEBOOK", "mlflow.traceName": "LangGraph", "mlflow.user": "yuzhang@niagarawater.com", "mlflow.artifactLocation": "dbfs:/databricks/mlflow-tracking/3530709779271609/tr-c415f30a45b748e6b729908a00e495c4/artifacts"}, "assessments": [{"name": "relevance_to_query", "source": {"source_type": "LLM_JUDGE", "source_id": "databricks"}, "trace_id": "tr-c415f30a45b748e6b729908a00e495c4", "expectation": null, "feedback": {"value": "yes", "error": null}, "rationale": "The answer pro

## Perform pre-deployment validation of the agent
Before registering and deploying the agent, we perform pre-deployment checks via the [mlflow.models.predict()](https://mlflow.org/docs/latest/python_api/mlflow.models.html#mlflow.models.predict) API. See [documentation](https://docs.databricks.com/machine-learning/model-serving/model-serving-debug.html#validate-inputs) for details

In [0]:
mlflow.models.predict(
    model_uri=f"runs:/{logged_agent_info.run_id}/agent",
    input_data={"messages": [{"role": "user", "content": "Hello!"}]},
    env_manager="uv",
)

2025/06/09 18:15:54 INFO mlflow.models.flavor_backend_registry: Selected backend for flavor 'python_function'
2025/06/09 18:15:56 INFO mlflow.utils.virtualenv: Creating a new environment in /tmp/virtualenv_envs/mlflow-a33d40a0598223584ba092d96d37c65abed4872b with python version 3.10.12 using uv
Using CPython 3.10.12 interpreter at: /usr/bin/python3.10
Creating virtual environment at: /tmp/virtualenv_envs/mlflow-a33d40a0598223584ba092d96d37c65abed4872b
Activate with: source /tmp/virtualenv_envs/mlflow-a33d40a0598223584ba092d96d37c65abed4872b/bin/activate
2025/06/09 18:15:58 INFO mlflow.utils.virtualenv: Installing dependencies
Using Python 3.10.12 environment at: /tmp/virtualenv_envs/mlflow-a33d40a0598223584ba092d96d37c65abed4872b
Resolved 3 packages in 100ms
Prepared 3 packages in 225ms
Installed 3 packages in 49ms
 + pip==22.3.1
 + setuptools==80.9.0
 + wheel==0.38.4
Using Python 3.10.12 environment at: /tmp/virtualenv_envs/mlflow-a33d40a0598223584ba092d96d37c65abed4872b
Resolved 176 

{"messages": [{"role": "assistant", "content": "Hello! I'd be happy to help you find places that cater to the needs of pregnant women. To provide you with the best recommendations, could you share some details about what you're looking for? For example:\n\n1. What type of place are you interested in? (restaurant, shop, medical center, spa, etc.)\n2. When would you like to visit? (specific time of day or day of week)\n3. Where are you located? (zipcode or address for distance calculation)\n4. Any specific pregnancy-related needs or preferences?\n\nOnce you provide these details, I can create a tailored recommendation request for you!", "id": "run--8e109d6e-c69b-4668-888e-f72694c1ae54-0"}]}

## Register the model to Unity Catalog

Update the `catalog`, `schema`, and `model_name` below to register the MLflow model to Unity Catalog.

In [0]:
mlflow.set_registry_uri("databricks-uc")

# TODO: define the catalog, schema, and model name for your UC model
catalog = "workspace"
schema = "default"
model_name = "pregancypal_start_conversation"
UC_MODEL_NAME = f"{catalog}.{schema}.{model_name}"

# register the model to UC
uc_registered_model_info = mlflow.register_model(
    model_uri=logged_agent_info.model_uri, name=UC_MODEL_NAME
)

Successfully registered model 'workspace.default.pregancypal_start_conversation'.
Created version '1' of model 'workspace.default.pregancypal_start_conversation'.


## Deploy the agent

In [0]:
from databricks import agents
agents.deploy(UC_MODEL_NAME, uc_registered_model_info.version, tags = {"endpointSource": "playground"})

2025/06/09 18:21:35 INFO mlflow.pyfunc: Validating input example against model signature
Successfully registered model 'workspace.default.feedback'.
Created version '1' of model 'workspace.default.feedback'.



    Deployment of workspace.default.pregancypal_start_conversation version 1 initiated.  This can take up to 15 minutes and the Review App & Query Endpoint will not work until this deployment finishes.

    View status: https://dbc-d2dc33b8-a5c3.cloud.databricks.com/ml/endpoints/agents_workspace-default-pregancypal_start_conversation
    Review App: https://dbc-d2dc33b8-a5c3.cloud.databricks.com/ml/review-v2/ebd6227794554b63ab37990406c2bbb7/chat
    Monitor: https://dbc-d2dc33b8-a5c3.cloud.databricks.com/ml/experiments/3530709779271609/evaluation-monitoring


Deployment(model_name='workspace.default.pregancypal_start_conversation', model_version='1', endpoint_name='agents_workspace-default-pregancypal_start_conversation', served_entity_name='workspace-default-pregancypal_start_conversation_1', query_endpoint='https://dbc-d2dc33b8-a5c3.cloud.databricks.com/serving-endpoints/agents_workspace-default-pregancypal_start_conversation/served-models/workspace-default-pregancypal_start_conversation_1/invocations', endpoint_url='https://dbc-d2dc33b8-a5c3.cloud.databricks.com/ml/endpoints/agents_workspace-default-pregancypal_start_conversation', review_app_url='https://dbc-d2dc33b8-a5c3.cloud.databricks.com/ml/review-v2/ebd6227794554b63ab37990406c2bbb7/chat')

## Next steps

After your agent is deployed, you can chat with it in AI playground to perform additional checks, share it with SMEs in your organization for feedback, or embed it in a production application. See [docs](https://docs.databricks.com/generative-ai/deploy-agent.html) for details